In [68]:
# Set the notebook display method and libraries
# inline = embedded plots, notebook = interactive plots
%matplotlib notebook
import os
import argparse
import numpy as np
from plantcv import plantcv as pcv
import matplotlib.pyplot as plt

In [69]:
def plot(image):
    plt.imshow(image)
    return plt.show()

In [70]:
# INPUT VARIABLES
## The options class mimics the workflow command-line argument parser that is used for workflow 
## parallelization. Using it while developing a workflow in Jupyter makes it easier to convert the workflow to a script later.
class options:
    def __init__(self):
        # Input image path/filename
        # self.image = "C:/Users/jcard/OneDrive - University of Georgia/kinect_imaging/scripts_pykinectazure/my_scripts/rgb_imgs/tray_1/T01_GH13_JC01_Jan-27-2023_1738_rgb.jpg"
        "------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------"
        # When running this code on your machine(or any Unix-based system) run the above line of code, I need to use the other formatting for filepaths since Windows uses \ instead of /
        self.image = "..\\image_processing\\rgb_images\\tray_1\\raw_images\\T01_GH13_JC01_Jan-25-2023_0832_rgb.jpg"
        # Debug mode = None, "plot", or "print"
        self.debug = "plot"
        # Store output images (True/False)
        self.writeimg = False
        # Results path/filename
        self.result = "results_seedling.csv"
        # Image output directory path
        self.outdir = "."
        
## args holds the input variables
args = options()
## Set debug to the global parameter
pcv.params.debug = args.debug

# Increase text size and thickness to make labels clearer
# (size may need to be altered based on original image size)
pcv.params.text_size = 20
pcv.params.text_thickness = 10

In [71]:
# Inputs:
# filename = Image file to be read in 
# mode = How to read in the image; either 'native' (default), 
# 'rgb', 'gray', 'csv', or 'envi

raw_img, path, filename = pcv.readimage(filename=args.image)
plt.imshow(raw_img)
plt.show()

<IPython.core.display.Javascript object>

In [72]:
# To improve colorspaces visualization, cut the image in such a way that just light blue background is visible. 
# (X and Y are starting X,Y coordinate respectively)
# h = y_axis total lenght , w = x_axis total lenght

img = pcv.crop(img=raw_img, x=45, y=80, h=885, w=1850)
plot(img)

<IPython.core.display.Javascript object>

In [57]:

# Visualize colorspaces
## The visualization tool converts the color image into HSV and LAB colorspaces and displays the 
## grayscale channels in a matrix so that they can be visualized simultaneously. The idea is to 
## select a channel that maximizes the difference between the plant and the background pixels.

colorspaces = pcv.visualize.colorspaces(rgb_img=img, original_img=False)

<IPython.core.display.Javascript object>

In [58]:
# Convert the color image to grayscale
## Converts the input color image into the LAB colorspace and returns the A (green-magenta) channel as a grayscale image.

# Inputs:
#   rbg_img = original image
#   channel = desired colorspace ('l', 'a', or 'b')
h_color = pcv.rgb2gray_hsv(rgb_img=img, channel='h')

<IPython.core.display.Javascript object>

In [59]:
# Visualize the distribution of grayscale values
## A histogram can be used to visualize the distribution of values in an image. The histogram can aid in the 
## selection of a threshold value.

## For this image, the large peak between 100-140 are from the brighter background pixels. 
## The smaller peak between 80-90 are the darker plant pixels.

# Inputs:
#   img         = a color or grayscale image
#   mask        = None (default), or mask
#   bins        = 100 (default) or number of desired number of evenly spaced bins
#   lower-bound = None (default) or minimum value on x-axis
#   upper-bound = None (default) or maximum value on x-axis
#   title       = None (default) or custom plot title
#   hist_data   = False (default) or True (if frequency distribution data is desired)
hist = pcv.visualize.histogram(img= h_color, bins=25)

<IPython.core.display.Javascript object>

In [60]:
# Threshold the grayscale image
## Use a threshold function (binary in this case) to segment the grayscale image into plant (white) and 
## background (black) pixels. Using the histogram above, a threshold point between 90-110 will segment the plant 
## and background peaks. Because the plants are the darker pixels in this image, use object_type="dark" to do an inverse threshold.

s_thresh = pcv.threshold.binary(gray_img=h_color, threshold=50, max_value=255, object_type='dark')

<IPython.core.display.Javascript object>

In [61]:
# Remove small background noise
## Thresholding mostly labeled plant pixels white but also labeled small regions of the background white.
## The fill function removes "salt" noise from the background by filtering white regions by size.

# Inputs:
#   bin_img = Binary image data
#   size    = minimum object area size in pixels (integer), smaller objects will be filled
a_fill = pcv.fill(bin_img=s_thresh, size=100)

<IPython.core.display.Javascript object>

In [62]:
# Section 3: Measure Individual Plants
## Need a completed binary mask

# Identify the outlines of all plants
## The binary mask (all values are either white or black) that resulted from thresholding and filtering 
## the thresholded image is used to identify the polygons that define the outlines of every connected white region. 
## Objects (or contours) can be nested, so a hierarchy that defines the relationship between objects is also calculated.

# Inputs:
#   img  = input image
#   mask = a binary mask used to detect objects
obj, obj_hierarchy = pcv.find_objects(img=img, mask=a_fill)
len(obj)

<IPython.core.display.Javascript object>

18

In [63]:
# Define a region of interest for each plant
## Use the multi-ROI tool to define a region of interest (ROI) for each pot in the tray. 
## Each ROI will be associated with a plant later. The ROIs do not need to completely contain 
## a whole plant but must only overlap a single plant each.

# Inputs:
#   img     = input image
#   coord   = top left coordinate to begin the ROI grid
#   radius  = radius for each ROI
#   spacing = spacing between each ROI
#   nrows   = number of rows in the ROI grid
#   ncols   = number of columns in the ROI grid
rois, roi_hierarchy = pcv.roi.multi(img=raw_img, coord=(168,205), radius=70, 
                                    spacing=(159, 161), nrows=5, ncols=11)

<IPython.core.display.Javascript object>

In [64]:
# Create a unique ID for each plant
## Create a sequence of values to label each plant within the image based on the ROI IDs.

# Inputs:
#   start = beginning value for range
#   stop  = ending value for range (exclusive)
plant_ids = range(0, len(rois))

In [65]:
# Measure each plant
## To measure each plant, iterate over the ROIs so that only one region (pot) is considered at a time. Within the loop, several steps are done:

# Subset the ith ROI, the corresponding hierarchy, and the plant ID
# Subset the objects found above that overlap the ith ROI
# Consolidate the contours the define a single plant into one object (composition)
# Analyze the plant shape and size characteristics

# Create a copy of the original image for annotations

# Inputs:
#   img = rgb image
img_copy = np.copy(img)

# Set debug to None (plotting all of the images would be very verbose output)
pcv.params.debug = None

# Create a for loop to interate through every ROI (plant) in the image
for i in range(0, len(rois)):
    # The ith ROI, ROI hierarchy, and plant ID
    roi = rois[i]
    hierarchy = roi_hierarchy[i]
    plant_id = plant_ids[i]
    # Subset objects that overlap the ROI
    # Inputs:
    #   img            = input image
    #   roi_contour    = a single ROI contour
    #   roi_hierarchy  = a single ROI hierarchy
    #   object_contour = all objects detected in a binary mask
    #   obj_hierarchy  = all object hierarchies
    #   roi_type       = "partial" (default) keeps contours that overlap
    #                    or are contained in the ROI. "cutto" cuts off
    #                    contours that fall outside the ROI. "largest"
    #                    only keeps the largest object within the ROI
    plant_contours, plant_hierarchy, mask, area = pcv.roi_objects(img=img, 
                                                                  roi_contour=roi, 
                                                                  roi_hierarchy=hierarchy, 
                                                                  object_contour=obj, 
                                                                  obj_hierarchy=obj_hierarchy, 
                                                                  roi_type="partial")

    # If the plant area is zero then no plant was detected for the ROI
    # and no measurements can be done
    if area > 0:
        # Combine contours together for each plant
        # Inputs:
        #   img       = input image
        #   contours  = contours that will be consolidated into a single object
        #   hierarchy = the relationship between contours
        plant_obj, plant_mask = pcv.object_composition(img=img, 
                                                       contours=plant_contours, 
                                                       hierarchy=plant_hierarchy)        
        # Analyze the shape of each plant
        # Inputs:
        #   img   = input image
        #   obj   = composed object contours
        #   mask  = binary mask that contours were derived from
        #   label = a label for the group of measurements (default = "default")
        img_copy = pcv.analyze_object(img=img_copy, obj=plant_obj, 
                                      mask=plant_mask, label=f"plant{plant_id}")

pcv.plot_image(img_copy)

<IPython.core.display.Javascript object>

In [66]:
# Save the results
## During analysis, measurements are stored in the background in the outputs recorder.

## This example includes image analysis for 'area', 'convex_hull_area', 'solidity', 'perimeter', 
## 'width', 'height', 'longest_path', 'center_of_mass, 'convex_hull_vertices', 'object_in_frame', 
## 'ellipse_center', 'ellipse_major_axis', 'ellipse_minor_axis', 'ellipse_angle', 'ellipse_eccentricity' using anayze_object.

## If other functions, for example color analysis, are desired, these should be included in the for loop.

## Here, results are saved to a CSV file for easy viewing, but when running workflows in parallel, save results as "json"

# Inputs:
#   filename  = filename for saving results
#   outformat = output file format: "json" (default) hierarchical format
#                                   or "csv" tabular format
pcv.outputs.save_results(filename=args.result, outformat="csv")

In [67]:
plant_area = pcv.outputs.observations['default']['pixel_area']['value']

KeyError: 'default'